In [34]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw/combined_data.csv')

In [16]:
df.head(10)

,Date,Time,Code,Value
0,05-20-1991,08:00,58,101
1,05-20-1991,08:00,33,5
2,05-20-1991,08:00,34,27
3,05-20-1991,12:00,60,89
4,05-20-1991,12:00,33,3
5,05-20-1991,18:00,62,94
6,05-20-1991,18:00,33,6
7,05-20-1991,22:00,48,109
8,05-21-1991,08:00,58,174
9,05-21-1991,08:00,33,5


In [4]:
df.sample(5)

,Date,Time,Code,Value
6942,04-12-1991,12:00,33,3
12148,06-01-1991,21:29,33,2
18704,03-08-1989,18:00,62,133
3127,04-25-1989,06:15,58,199
20621,08-12-1990,18:13,33,7


In [5]:
df.shape

(29330, 4)

In [6]:
df.columns.tolist()

['Date', 'Time', 'Code', 'Value']

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29330 entries, 0 to 29329
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    29297 non-null  object
 1   Time    29330 non-null  object
 2   Code    29330 non-null  int64 
 3   Value   29297 non-null  object
dtypes: int64(1), object(3)
memory usage: 916.7+ KB


## Removing the null values

In [8]:
df.nunique()

Date     1141
Time     1311
Code       24
Value     696
dtype: int64

In [9]:
df.isnull().sum()

Date     33
Time      0
Code      0
Value    33
dtype: int64

In [10]:
df.isnull().mean() * 100
# As we have only 33 or 0.1% null values in our total dataset so we can drop them

Date     0.112513
Time     0.000000
Code     0.000000
Value    0.112513
dtype: float64

In [11]:
df.dropna(inplace=True)

In [17]:
df.isna().sum()

Date     0
Time     0
Code     0
Value    0
dtype: int64

In [19]:
len(df)

29264

In [24]:
df[["Date", "Time"]]

,Date,Time
0,05-20-1991,08:00
1,05-20-1991,08:00
2,05-20-1991,08:00
3,05-20-1991,12:00
4,05-20-1991,12:00
...,...,...
29325,09-25-1990,12:08
29326,09-25-1990,15:49
29327,09-25-1990,20:05
29328,09-25-1990,20:07


In [26]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')

In [31]:
df

,Date,Time,Code,Value,datetime
0,05-20-1991,08:00,58,101,1991-05-20 08:00:00
1,05-20-1991,08:00,33,5,1991-05-20 08:00:00
2,05-20-1991,08:00,34,27,1991-05-20 08:00:00
3,05-20-1991,12:00,60,89,1991-05-20 12:00:00
4,05-20-1991,12:00,33,3,1991-05-20 12:00:00
...,...,...,...,...,...
29325,09-25-1990,12:08,33,3,1990-09-25 12:08:00
29326,09-25-1990,15:49,64,56,1990-09-25 15:49:00
29327,09-25-1990,20:05,63,327,1990-09-25 20:05:00
29328,09-25-1990,20:07,33,5,1990-09-25 20:07:00


In [37]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.weekday

df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

In [40]:
df

,Date,Time,Code,Value,datetime,year,month,day,hour,weekday,hour_sin,hour_cos
0,05-20-1991,08:00,58,101,1991-05-20 08:00:00,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
1,05-20-1991,08:00,33,5,1991-05-20 08:00:00,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
2,05-20-1991,08:00,34,27,1991-05-20 08:00:00,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
3,05-20-1991,12:00,60,89,1991-05-20 12:00:00,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
4,05-20-1991,12:00,33,3,1991-05-20 12:00:00,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
29325,09-25-1990,12:08,33,3,1990-09-25 12:08:00,1990.0,9.0,25.0,12.0,1.0,1.224647e-16,-1.000000
29326,09-25-1990,15:49,64,56,1990-09-25 15:49:00,1990.0,9.0,25.0,15.0,1.0,-7.071068e-01,-0.707107
29327,09-25-1990,20:05,63,327,1990-09-25 20:05:00,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000
29328,09-25-1990,20:07,33,5,1990-09-25 20:07:00,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000


In [ ]:
# df.to_csv('../data/processed/cleaned_data.csv', index=False)

In [44]:
df.drop(["Date","datetime", "Time"], axis=1, inplace=True)

In [45]:
df

,Code,Value,year,month,day,hour,weekday,hour_sin,hour_cos
0,58,101,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
1,33,5,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
2,34,27,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
3,60,89,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
4,33,3,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
...,...,...,...,...,...,...,...,...,...
29325,33,3,1990.0,9.0,25.0,12.0,1.0,1.224647e-16,-1.000000
29326,64,56,1990.0,9.0,25.0,15.0,1.0,-7.071068e-01,-0.707107
29327,63,327,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000
29328,33,5,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000


In [46]:
df.dtypes

Code          int64
Value        object
year        float64
month       float64
day         float64
hour        float64
weekday     float64
hour_sin    float64
hour_cos    float64
dtype: object

In [51]:
df["Value"] = pd.to_numeric(df['Value'], errors='coerce')
df

,Code,Value,year,month,day,hour,weekday,hour_sin,hour_cos
0,58,101.0,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
1,33,5.0,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
2,34,27.0,1991.0,5.0,20.0,8.0,0.0,8.660254e-01,-0.500000
3,60,89.0,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
4,33,3.0,1991.0,5.0,20.0,12.0,0.0,1.224647e-16,-1.000000
...,...,...,...,...,...,...,...,...,...
29325,33,3.0,1990.0,9.0,25.0,12.0,1.0,1.224647e-16,-1.000000
29326,64,56.0,1990.0,9.0,25.0,15.0,1.0,-7.071068e-01,-0.707107
29327,63,327.0,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000
29328,33,5.0,1990.0,9.0,25.0,20.0,1.0,-8.660254e-01,0.500000


In [52]:
df.dtypes

Code          int64
Value       float64
year        float64
month       float64
day         float64
hour        float64
weekday     float64
hour_sin    float64
hour_cos    float64
dtype: object

In [56]:
len(df)

29264

In [54]:
df.to_csv('../data/processed/cleaned_data.csv', index=False)